# Merging Datasets

**Author:** Marang Mutloatse

**Description:** This script prepares Patient data and merges the relevant dataset to it

**Version:** 0.0.1

**Status:** Development

In [1]:
import pandas as pd
import yaml
import datetime as dt
import warnings
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
import glob
import os
from datetime import timedelta, date
plt.style.use('fivethirtyeight')
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

# Working Functions

In [2]:
def load_excel_sheet(path: str):
    if path is not None:
        try:
            xls = pd.ExcelFile(path)
            df = xls.parse(skiprows=0)
            return df
        except Exception as e:
            print(f"Exception on loading excel spreadsheet with error: {e}")

# Loading Data

In [3]:
user_dev_path = os.path.dirname(os.getcwd())
try: 
    with open (user_dev_path + '/' + 'config_LTFU.yaml', 'r') as file:
        config = yaml.safe_load(file)
except Exception as e:
    print(f'Error reading the config file, {e}')

In [4]:
input_root = config['rise_files']['raw_data_path']
patient_file = config['rise_files']['raw_patient_file']
pharmacy_file = config['rise_files']['raw_pharmacy_file']
lab_file = config['rise_files']['raw_lab_file']
clinic_file = config['rise_files']['raw_clinic_file']
patient_input = input_root + patient_file
pharmacy_input = input_root + pharmacy_file
lab_input = input_root + lab_file
clinic_input = input_root + clinic_file

In [5]:
print(patient_input)
print(pharmacy_input)
print(lab_input)
print(clinic_input)

/data/rise_data/PatientDemographicsData.xlsx
/data/rise_data/concat_pharmacy_data.csv
/data/rise_data/All LaboratoryData_Flat File.csv
/data/rise_data/ClinicData.xlsx


In [6]:
df = load_excel_sheet(patient_input)

In [11]:
# list datetime columns
DATE_COLS = [
    "DATE_BIRTH",
    "DATE_STARTED",
    "DATE_REGISTRATION",
    "DATE_CONFIRMED_HIV",
    "DATE_LAST_REFILL",
    "DATE_NEXT_REFILL",
    "DATE_CURRENT_STATUS",
    "DATE_LAST_CLINIC",
    "DATE_NEXT_CLINIC",
    "DATE_LAST_CD4",
    "DATE_LAST_VIRAL_LOAD",
    "VIRAL_LOAD_DUE_DATE",
    "DATE_ENROLLED_PMTCT"
]
# Convert to datetime format
df.loc[:,DATE_COLS] = df.loc[:,DATE_COLS].apply(pd.to_datetime,errors = "coerce")

## Data Preparation

### Age Prep

In [13]:
conditions = [(df['age_unit'] =='month(s)'),(df['age_unit'] == 'day(s)'),(df['age_unit'] == 'year(s)')]
values = [(df['AGE']/12),(df['AGE']/365),df['AGE']]
df['EF_CURRENT_AGE'] = np.select(conditions,values)

In [14]:
date_today = date.today()
date_today = pd.to_datetime(date_today)
time_of_day = pd.Series(np.full(len(df[df['DATE_BIRTH'].isnull()]),date_today))

In [15]:
df['DATE_BIRTH']=np.where(df['DATE_BIRTH'].isnull(), (time_of_day - (pd.to_timedelta((df['EF_CURRENT_AGE']*365),unit='d'))),df['DATE_BIRTH'])

In [16]:
df['DATE_BIRTH'].isnull().sum()

20

In [17]:
df = df.rename(columns={'STATE':'STATE_RESIDENCE'})
df.columns = df.columns.str.upper()
df = df.rename(columns={'FACILITY NAME': 'FACILITY_NAME','LGA': 'EF_LGA_RESIDENCE'})

### Data Cleaning

In [18]:
standardized_col = [
    "MARITAL_STATUS",
    "OCCUPATION",
    "EDUCATION",
    "ENTRY_POINT",
    "STATUS_REGISTRATION",
    "ENROLLMENT_SETTING",
    "CURRENT_STATUS",
    "CAUSE_DEATH",
    "WARD",
    "TB_STATUS",
    "STATE_RESIDENCE",
    "EF_LGA_RESIDENCE",
]

In [19]:
# Grouping entries by first lowering, then capitalizng all word
for col in standardized_col:
    df[col] = df[col].str.title()

In [21]:
education_dict = {
    "None": "None",
    "Junior Secondary": "Junior Secondary",
    "Senior Secondary": "Senior Secondary",
    "Post Secondary": "Post Secondary",
    "Quranic":"Quranic",
    "Quranic Education": "Quranic",
    "Quranic Education,Quranic Education": "Quranic",
    "Senior Secondary,Senior Secondary": "Senior Secondary",
    "Post Secondary,Post Secondary": "Post Secondary",
}

marital_status_dict = {
    "Married": "Married",
    "Single": "Single",
    "Widowed": "Widowed",
    "Divorced": "Divorced",
    "Separated": "Separated",
    "Seperated": "Separated",
    "Windowed": "Widowed",
}

enrollment_setting_dict = {
    "Facility":"Facility",
    "Community":"Community",
    "Community Based Organisation": "Community",
    "Clinical Platforms (Community Pharmacy)":"Community",
    "Clinical Platforms (Chemists/Pmvs/Dispensary)":"Clinical Platforms",
    "Clinical Platforms (Phcs/Private Clinics/Nursing Homes)":"Clinical Platforms",
    "Clinical Platforms (Laboratories)":"Clinical Platforms",
    "Clinical Platforms (Tbas)":"Clinical Platforms",
}

last_clinic_stage_dict = {
    "Stage I":"Stage I",
    "Stage II":"Stage II",
    "Stage III":"Stage III",
    "Stage II?":"Stage II",
    "Stage IIt":"Stage III",
    "Stage IV": "Stage IV",   
}

status_registration_dict = {
    "Hiv+ Non Art":"Hiv+ Non Art",
    "Art Transfer In":"Art Transfer In",
    "Pre-Art Transfer In":"Pre-Art Transfer In",
    "Hiv Exposed Status Unknown":"Hiv Exposed Status Unknown",
    "Hiv Exposed Infant Status Unknown":"Hiv Exposed Status Unknown",
    "Art Start - External":"Art Start - External",
    "Hiv Negative":"Hiv Negative",
    "Hiv Exposed Infant Status Negative":"Hiv Negative",
}

entry_point_dict = {
    "Hct":"Hct",
    "Outreach":"Outreach",
    "Outreaches":"Outreach",
    "In-Patient":"In Patient",
    "In Patients":"In Patient",
    "Opd":"Opd",
    "Pmtct":"Pmtct",
    "Anc/Pmtct":"Pmtct",
    "Cbo":"Cbo",
    "Others":"Others",
    "Transfer-In":"Transfer-In"
    "Tb Dots":"Others",
    "Sti Clinic":"Others",
}
# tb_status_dict = {"No Sign Or Symptoms Of Tb":"No Sign Or Symptoms Of Tb",
#                  "No Signs Or Symptoms Of Tb":"No Sign Or Symptoms Of Tb",
#                  "Currently On Inh Prophylaxis":"Currently On Inh Prophylaxis",
#                  "Currently On Inh Prophylaxis,Currently On Inh Prophylaxis": "Currently On Inh Prophylaxis",
#                  }

In [22]:
df['EDUCATION'] = df['EDUCATION'].map(education_dict)
df['MARITAL_STATUS'] = df['MARITAL_STATUS'].map(marital_status_dict)
df['ENROLLMENT_SETTING'] = df['ENROLLMENT_SETTING'].map(enrollment_setting_dict)
df['LAST_CLINIC_STAGE'] = df['LAST_CLINIC_STAGE'].map(last_clinic_stage_dict)
df['STATUS_REGISTRATION'] = df['STATUS_REGISTRATION'].map(status_registration_dict)
df['ENTRY_POINT'] = df['ENTRY_POINT'].map(entry_point_dict)

### Dealing with Missing Values

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(missing_values=np.NaN,strategy='most_frequent') 

In [ ]:
df["STATE_RESIDENCE"] = df.groupby(["STATE", "L.G.A", "FACILITY_NAME"], sort=False)[
    "STATE_RESIDENCE"
].apply(lambda x: x.fillna(x.mode().iloc[0]))

df["EF_LGA_RESIDENCE"] = df.groupby(["STATE", "L.G.A", "FACILITY_NAME"], sort=False)[
    "EF_LGA_RESIDENCE"
].apply(lambda x: x.fillna(x.mode().iloc[0]))

df["ENROLLMENT_SETTING"] = df.groupby(
    ["STATE","L.G.A","FACILITY_NAME"], sort=False)["ENROLLMENT_SETTING"].apply(lambda x: x.fillna(x.mode().iloc[0]))

df["ENROLLED_OVC"] = df["ENROLLED_OVC"].fillna(0.0)
df["BIOMETRIC"] = df["BIOMETRIC"].fillna(0.0)
df["STATUS_REGISTRATION"] = pd.DataFrame(
    imputer.fit_transform(df[["STATUS_REGISTRATION"]])
)
df["CURRENT_STATUS"] = pd.DataFrame(imputer.fit_transform(df[["CURRENT_STATUS"]]))
df["LAST_CLINIC_STAGE"] = df.groupby(["CURRENT_STATUS"], sort=False)[
    "LAST_CLINIC_STAGE"
].apply(lambda x: x.fillna(x.mode().iloc[0]))

df["MARITAL_STATUS"] = df["MARITAL_STATUS"].fillna("Unknown")
df["OCCUPATION"] = df["OCCUPATION"].fillna("Unknown")
df["EDUCATION"] = df["EDUCATION"].fillna("Unknown")

In [ ]:
education_dict_label = {
    "None": 0,
    "Junior Secondary": 1,
    "Senior Secondary": 2,
    "Post Secondary": 3,
    "Quranic":2,
}

last_clinic_stage_dict_label = {
    "Stage I":0,
    "Stage II":1,
    "Stage III":2,
    "Stage IV":3,   
}

occupation_dict_label = {
    "Unemployed":0,
    "Unknown":0,
    "Student":1,
    "Employed":2,
    "Retired":3,
}

## Age and Gender Check

In [ ]:
df_AGE =  df.query("EF_CURRENT_AGE < 100")
ax_age = sns.histplot(data=df_AGE, x="EF_CURRENT_AGE", binwidth=10)
ax_age.set(xlabel='Age', ylabel='Number of Patients')
ax_age.set_title("Age Distribution")

In [ ]:
df_pie = df.groupby("GENDER")["PATIENT_ID"].count()
pie, ax = plt.subplots(figsize=[10,6])
labels = df_pie.keys()
plt.pie(x=df_pie, autopct="%.1f%%", labels=labels, pctdistance=0.5)
plt.title("Number of Patients: By Gender", fontsize=14)

## One Hot Encoding

### Columns to Join

In [ ]:
col_list = ["PATIENT_ID",
"HOSPITAL_NUM",
"GENDER",
"DATE_BIRTH",
"EF_CURRENT_AGE",
"MARITAL_STATUS",
"OCCUPATION",
"EDUCATION",
"ENTRY_POINT",
"STATUS_REGISTRATION",
"PREGNANT",
"BREASTFEEDING",
"DATE_REGISTRATION",
"DATE_STARTED",
"ENROLLMENT_SETTING",
"DATE_CONFIRMED_HIV",
"ENROLLED_OVC",
"LAST_CLINIC_STAGE",
"LAST_VIRAL_LOAD",
"LAST_CD4",
"LAST_CD4P",
"VIRAL_LOAD_DUE_DATE",
"VIRAL_LOAD_TYPE",
"CURRENT_STATUS",
"DATE_CURRENT_STATUS",
"DATE_LAST_CLINIC",
"TB_STATUS",
"DATE_LAST_VIRAL_LOAD"]

In [ ]:
# Replace spaces in columns names
df.columns = df.columns.str.replace(' ','_')

# Standardizing columns
df.columns = map(lambda x: str(x).upper(), df.columns)

In [ ]:
drops_cols = ['PARTNERINFORMATION_ID','SOURCE_REFERRAL','TIME_HIV_DIAGNOSIS','AGREED_DATE','CAUSE_DEATH']

In [ ]:
df = df.add_prefix('')

In [ ]:
break

In [ ]:
df_merge = df.loc[:,col_list]

In [ ]:
df_merge = df_merge.add_prefix('P_')

# Join Datasets

## Pharmacy Merging data

In [ ]:
df_pharm = pd.read_csv(input_root+'concat_pharmacy_data.csv',low_memory=False)

In [ ]:
df_pharm['PATIENT_ID'] = df_pharm['PATIENT_ID'].astype(str)
df_merge['P_PATIENT_ID'] = df_merge['P_PATIENT_ID'].astype(str)

In [ ]:
df_merged_pharm = pd.merge(df_pharm,df_merge,left_on="PATIENT_ID",right_on="P_PATIENT_ID",how="left")

In [ ]:
df_merged_pharm.to_csv(input_root+'pharmacy_merged.csv',index=False,header=True)

## Laboratory File

In [ ]:
df_lab = pd.read_csv(lab_input,low_memory=False)

In [ ]:
df_lab['PATIENT_ID'] = df_lab['PATIENT_ID'].astype(str)
df_merged_lab = pd.merge(df_lab,df_merge,left_on="PATIENT_ID",right_on="P_PATIENT_ID",how="left")

In [ ]:
df_merged_lab.to_csv(input_root+'lab_merged.csv',index=False,header=True)

## Clinic File

In [ ]:
df_clinic = load_excel_sheet(clinic_input)

In [ ]:
df_clinic['PATIENT_ID'] = df_clinic['PATIENT_ID'].astype(str)
df_merged_clinic = pd.merge(df_clinic,df_merge,left_on="PATIENT_ID",right_on="P_PATIENT_ID",how="left")

In [ ]:
df_merged_clinic.to_csv(input_root+'clinic_merged.csv',index=False,header=True)